# 1. Loading Data

In [1]:
import sys
print(sys.executable)


c:\Users\julia\Documents\ds-personal-projects\customer-churn-prediction\.venv\Scripts\python.exe


In [2]:
import pandas as pd

DATA_PATH = "../data/raw/telco_churn.csv"
df = pd.read_csv(DATA_PATH)

df.head(), df.shape


(   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
 0  7590-VHVEG  Female              0     Yes         No       1           No   
 1  5575-GNVDE    Male              0      No         No      34          Yes   
 2  3668-QPYBK    Male              0      No         No       2          Yes   
 3  7795-CFOCW    Male              0      No         No      45           No   
 4  9237-HQITU  Female              0      No         No       2          Yes   
 
       MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
 0  No phone service             DSL             No  ...               No   
 1                No             DSL            Yes  ...              Yes   
 2                No             DSL            Yes  ...               No   
 3  No phone service             DSL            Yes  ...              Yes   
 4                No     Fiber optic             No  ...               No   
 
   TechSupport StreamingTV StreamingMovies      

# 2. Data Overview

# 3. Target Variable: Churn

# 4. Data Quality Checks

# 5. Numeric Features Analysis

# 6. Categorical Features Analysis

# 7. EDA Summary